<a href="https://colab.research.google.com/github/YasserJxxxx/Examen_Parcial/blob/main/Prototipo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import kagglehub
import os
import sys
from kagglehub import KaggleDatasetAdapter

# ====================================================================
# --- RUTAS DE ARCHIVOS PARA PERSISTENCIA ---
# ====================================================================
MODEL_PATH = "perceptron_model.npy"
SCALER_PATH = "scaler_params.npy"
TRAIN_DATA_PATH = "train_data.npz"

# ====================================================================
# --- 1. DEFINICIÓN DE LA CLASE PERCEPTRÓN (DESDE CERO) ---
# ====================================================================

class Perceptron:
    """Implementación del algoritmo del Perceptrón desde cero."""

    # 💥 Épocas predeterminadas AHORA son 500
    def __init__(self, n_features, n_epochs=500, learning_rate=0.5):
        # Inicialización de pesos con ceros. Para problemas no separables linealmente,
        # una inicialización aleatoria pequeña podría ser mejor, pero mantendremos ceros por simplicidad.
        self.weights = np.zeros(n_features + 1)
        self.n_epochs = n_epochs
        self.initial_learning_rate = learning_rate
        self.accuracy_history = []

    def _predict_raw(self, X):
        """Calcula la suma ponderada (dot product + bias)."""
        return np.dot(X, self.weights[1:]) + self.weights[0]

    def predict(self, X):
        """Función de activación (escalón): 1 si la salida raw >= 0, sino 0."""
        return np.where(self._predict_raw(X) >= 0.0, 1, 0)

    def _decay_learning_rate(self, epoch):
        """Programa de decaimiento: tasa de aprendizaje disminuye con las épocas."""
        # Tasa de decaimiento ajustada para 500 épocas.
        decay_rate = 0.005
        return self.initial_learning_rate / (1.0 + decay_rate * epoch)

    def train(self, X, y):
        """Entrena el modelo del Perceptrón."""

        print("\n--- INICIANDO ENTRENAMIENTO (500 ÉPOCAS) ---")

        for epoch in range(self.n_epochs):
            lr = self._decay_learning_rate(epoch)
            errors = 0

            for xi, target in zip(X, y):
                prediction = self.predict(xi)
                update = target - prediction

                if update != 0:
                    self.weights[1:] += lr * update * xi
                    self.weights[0] += lr * update
                    errors += 1

            # Mostrar métricas cada 50 épocas para no saturar la salida
            if (epoch + 1) % 50 == 0 or (epoch + 1) == self.n_epochs:
                y_pred_train = self.predict(X)
                current_accuracy = self.evaluate_metrics(y, y_pred_train)['Accuracy']
                self.accuracy_history.append(current_accuracy)

                print(f"Epoch {epoch+1}/{self.n_epochs}, Errores: {errors}, Tasa de Aprendizaje: {lr:.4f}, Precisión (Train): {current_accuracy:.4f}")

        print("--- ENTRENAMIENTO COMPLETADO ---")

    def evaluate_metrics(self, y_true, y_pred):
        """Calcula métricas de evaluación (Accuracy, Precision, Recall, F1-Score)."""

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # Matriz de Confusión
        TP = np.sum((y_pred == 1) & (y_true == 1))
        TN = np.sum((y_pred == 0) & (y_true == 0))
        FP = np.sum((y_pred == 1) & (y_true == 0))
        FN = np.sum((y_pred == 0) & (y_true == 1))

        # Métricas
        accuracy = (TP + TN) / len(y_true) if len(y_true) > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

        return {
            "Accuracy": accuracy,
            "Precision (Fraude)": precision,
            "Recall (Fraude)": recall,
            "F1-Score (Fraude)": f1_score,
            "Specificity (Legítima)": specificity,
            "Matriz de Confusión": {"TP": TP, "TN": TN, "FP": FP, "FN": FN}
        }

# ====================================================================
# --- 2. PREPROCESAMIENTO Y DIVISIÓN ---
# ====================================================================

def preprocess_and_split(df, test_size=0.3, random_state=42):
    """Gestión de nulos, selección, escalado (desde cero) y división de datos."""

    print("Iniciando Preprocesamiento y División de Datos...")

    # A. Gestión de Valores Nulos (Verificación/Imputación)
    if df.isnull().sum().any():
        print("¡Advertencia! Se han encontrado valores nulos. Imputando con la media.")
        df = df.fillna(df.mean())
    else:
        print("Verificación de Nulos: No se encontraron valores nulos. ✅")

    # B. Selección de Características (V1-V28 y Amount)
    features = [col for col in df.columns if col not in ['Time', 'Class']]
    X = df[features].values
    y = df['Class'].values

    # C. Normalización Min-Max (Desde Cero) - CRUCIAL para Perceptrón
    X_min = X.min(axis=0)
    X_max = X.max(axis=0)

    X_range = X_max - X_min
    X_range[X_range == 0] = 1

    X_scaled = (X - X_min) / X_range
    print("Normalización Min-Max 'desde cero' aplicada. 📊")

    # D. División del Dataset (70% Train, 30% Test)
    np.random.seed(random_state)
    n_total = len(X_scaled)
    n_train = int(n_total * (1 - test_size))

    indices = np.random.permutation(n_total)
    train_indices = indices[:n_train]
    test_indices = indices[n_train:]

    X_train, y_train = X_scaled[train_indices], y[train_indices]
    X_test, y_test = X_scaled[test_indices], y[test_indices]

    print(f"División completada. Train: {len(X_train)}, Test: {len(X_test)}")

    scaler_params = {'min': X_min, 'max': X_max, 'range': X_range}

    return X_train, X_test, y_train, y_test, features, scaler_params

# ====================================================================
# --- 3. PERSISTENCIA Y EVALUACIÓN ---
# ====================================================================

def save_model_and_data(perceptron_model, X_train, y_train, features, scaler_params):
    """Guarda el modelo entrenado y los datos de preprocesamiento."""

    np.save(MODEL_PATH, perceptron_model.weights)
    print(f"\nModelo (pesos) guardado en: {MODEL_PATH}")

    np.savez(TRAIN_DATA_PATH, X_train=X_train, y_train=y_train, features=features)
    print(f"Datos de entrenamiento guardados en: {TRAIN_DATA_PATH}")

    np.save(SCALER_PATH, scaler_params)
    print(f"Parámetros del escalador guardados en: {SCALER_PATH}")

def load_model_and_scaler(n_features):
    """Carga los pesos del modelo y los parámetros del escalador."""

    weights = np.load(MODEL_PATH, allow_pickle=True)
    scaler_params = np.load(SCALER_PATH, allow_pickle=True).item()

    # Usamos 500 épocas por defecto, pero el modelo cargado ignora este valor
    loaded_perceptron = Perceptron(n_features=n_features)
    loaded_perceptron.weights = weights

    print(f"\nModelo cargado desde {MODEL_PATH}. Entrenamiento omitido. 🚀")
    return loaded_perceptron, scaler_params

def show_evaluation(perceptron, X_test, y_test):
    """Muestra las métricas de evaluación del modelo."""

    print("\n" + "#"*70)
    print("EVALUACIÓN DEL MODELO EN EL CONJUNTO DE PRUEBA (30%)")
    print("#"*70)

    y_pred_test = perceptron.predict(X_test)
    metrics = perceptron.evaluate_metrics(y_test, y_pred_test)

    print("Métricas de Desempeño:")
    for key, value in metrics.items():
        if isinstance(value, dict):
            print(f"  {key}: {value}")
        else:
            print(f"  {key}: {value:.4f}")

    print("\n--- Análisis de Métricas (Clase Desequilibrada) ---")
    # El Recall es la capacidad para detectar FRAUDES REALES.
    print(f"El **Recall (Fraude): {metrics['Recall (Fraude)']:.4f}** es la métrica más importante, pues indica la capacidad del modelo para detectar fraudes reales (minimizar pérdidas).")

# ====================================================================
# --- 4. INSERCIÓN DE DATOS EXTERNOS (CON VALIDACIÓN) ---
# ====================================================================

def insert_and_test_data(model, scaler_params, features, X_test, y_test):
    """Permite al usuario ingresar y probar el modelo con datos externos, con validación."""

    n_expected = len(features)

    X_min = scaler_params['min']
    X_max = scaler_params['max']
    X_range = scaler_params['range']

    # Generar sugerencia de entrada
    sample_index = np.random.randint(0, len(X_test))
    random_sample = X_test[sample_index] * X_range + X_min
    random_sample_list = [f"{val:.4f}" for val in random_sample]

    print("\n" + "="*70)
    print("INSERCIÓN DE DATOS EXTERNOS PARA PREDICCIÓN")
    print("="*70)

    print(f"La transacción requiere {n_expected} valores (NO escalados), separados por **espacios**, en el orden:")
    print("-" * 70)
    print(f"COLUMNAS: {', '.join(features)}")
    print("-" * 70)

    print(f"Sugerencia de entrada (valores de una transacción aleatoria) [Clase Real: {y_test[sample_index]}]:")
    print(f"Valores Sugeridos: {' '.join(random_sample_list)}")

    while True:
        try:
            user_input_str = input("\nIntroduzca los 30 valores separados por espacios (o presione Enter para usar la sugerencia): ")

            if not user_input_str:
                input_data = random_sample
                print("Usando la muestra aleatoria para la predicción...")
                break

            input_list = user_input_str.split()
            input_data = np.array([float(x.strip()) for x in input_list])

            if len(input_data) != n_expected:
                print(f"❌ Error: Se esperaban {n_expected} valores, pero se introdujeron {len(input_data)}. Vuelva a intentarlo.")
                continue

            break

        except ValueError:
            print("❌ Error: Asegúrese de que todos los valores sean números válidos. Vuelva a intentarlo.")
        except Exception as e:
            print(f"❌ Error inesperado: {e}. Vuelva a intentarlo.")

    # Aplicar el mismo escalado Min-Max (CRUCIAL)
    X_external_scaled = (input_data - X_min) / X_range

    # Realizar la predicción
    prediction = model.predict(X_external_scaled.reshape(1, -1))

    # Mapear el resultado
    result = "FRAUDULENTA (1)" if prediction[0] == 1 else "LEGÍTIMA (0)"

    print("\n" + "-"*70)
    print(f"PREDICCIÓN FINAL: {result}")
    print("-" * 70)

# ====================================================================
# --- 5. SUBMENÚ PRINCIPAL Y FLUJO DE EJECUCIÓN ---
# ====================================================================

def main_menu(perceptron, X_test, y_test, features, scaler_params):
    """Muestra el menú de opciones y gestiona el flujo del programa."""

    while True:
        print("\n" + "="*70)
        print("MENU PRINCIPAL - DETECCIÓN DE FRAUDE CON PERCEPTRÓN")
        print("="*70)
        print(" [E]valuar modelo (usando 30% conjunto de prueba)")
        print(" [I]nsertar nuevos datos para predicción")
        print(" [S]alir")
        print("-" * 70)

        choice = input("Seleccione una opción (E/I/S): ").upper()

        if choice == 'E':
            show_evaluation(perceptron, X_test, y_test)
        elif choice == 'I':
            insert_and_test_data(perceptron, scaler_params, features, X_test, y_test)
        elif choice == 'S':
            print("Saliendo del programa. ¡Hasta luego! 👋")
            sys.exit(0)
        else:
            print("Opción no válida. Por favor, seleccione E, I o S.")


if __name__ == "__main__":

    # 1. ELIMINACIÓN DE ARCHIVOS ANTIGUOS para garantizar que el nuevo modelo con 500 épocas se entrene y se guarde correctamente.
    if os.path.exists(SCALER_PATH) and os.path.exists(MODEL_PATH):
        try:
            os.remove(SCALER_PATH)
            os.remove(MODEL_PATH)
            print("Archivos de persistencia antiguos eliminados para forzar el re-entrenamiento con 500 épocas.")
        except Exception as e:
            print(f"Advertencia: No se pudieron eliminar los archivos de persistencia. El programa continuará. {e}")


    # 2. Cargar el dataset
    print("Cargando dataset 'mlg-ulb/creditcardfraud' desde KaggleHub...")
    try:
        df = kagglehub.load_dataset(
            KaggleDatasetAdapter.PANDAS,
            "mlg-ulb/creditcardfraud",
            "creditcard.csv"
        )
        print(f"Dataset cargado con {len(df)} registros y {df.shape[1]} columnas.")
    except Exception as e:
        print(f"Error al cargar el dataset de KaggleHub. Verifique la conexión o dependencias. Error: {e}")
        sys.exit(1)

    # 3. Preprocesar y dividir los datos
    X_train, X_test, y_train, y_test, features, scaler_params = preprocess_and_split(df)
    n_features = X_train.shape[1]

    # 4. Lógica de Persistencia: Cargar o Entrenar (forzaremos el entrenamiento aquí)

    if os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH):
        # Esta rama solo se ejecuta si los archivos se guardaron después de la limpieza.
        perceptron, scaler_params = load_model_and_scaler(n_features)
    else:
        # Se ejecutará la primera vez después de la limpieza
        print("\nEl modelo no se encontró (o se forzó la eliminación). Realizando el entrenamiento con 500 épocas...")
        # Instanciamos con 500 épocas
        perceptron = Perceptron(n_features=n_features, n_epochs=500, learning_rate=0.5)
        perceptron.train(X_train, y_train)

        save_model_and_data(perceptron, X_train, y_train, features, scaler_params)

    # 5. Iniciar el Menú Principal
    main_menu(perceptron, X_test, y_test, features, scaler_params)

Archivos de persistencia antiguos eliminados para forzar el re-entrenamiento con 500 épocas.
Cargando dataset 'mlg-ulb/creditcardfraud' desde KaggleHub...


/tmp/ipython-input-3735135164.py:310: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'creditcardfraud' dataset.
Dataset cargado con 284807 registros y 31 columnas.
Iniciando Preprocesamiento y División de Datos...
Verificación de Nulos: No se encontraron valores nulos. ✅
Normalización Min-Max 'desde cero' aplicada. 📊
División completada. Train: 199364, Test: 85443

El modelo no se encontró (o se forzó la eliminación). Realizando el entrenamiento con 500 épocas...

--- INICIANDO ENTRENAMIENTO (500 ÉPOCAS) ---
Epoch 50/500, Errores: 248, Tasa de Aprendizaje: 0.4016, Precisión (Train): 0.9990
Epoch 100/500, Errores: 262, Tasa de Aprendizaje: 0.3344, Precisión (Train): 0.9991
Epoch 150/500, Errores: 252, Tasa de Aprendizaje: 0.2865, Precisión (Train): 0.9991
Epoch 200/500, Errores: 264, Tasa de Aprendizaje: 0.2506, Precisión (Train): 0.9992
Epoch 250/500, Errores: 258, Tasa de Aprendizaje: 0.2227, Precisión (Train): 0.9991
Epoch 300/500, Errores: 256, Tasa de Aprendizaje: 0.2004, Precisión (Train): 0.9991
Epoch 350/500, Errores: 2

KeyboardInterrupt: Interrupted by user